In [1]:
import pandas as pd
import numpy as np

In [117]:
PATH_ROSTER = "D:\\Run Analysis\\blob_storage\\roster\\35th_2021.xlsx"
PATH_SAL = "D:\\Run Analysis\\blob_storage\\driver_salary\\processed\\35th_2021.xlsx"
sal_df = pd.read_excel(PATH_SAL)
# Employee_ID	Hours	 Gross 	Super
roster_df = pd.read_excel(PATH_ROSTER)

sal_df.Employee_ID = sal_df.Employee_ID.str.lower()
sal_df['total'] = sal_df.Gross + sal_df.Super
count_pri_dr = roster_df.groupby(['Primary_employeeID'])['Date'].count().rename("pri_dr")

count_sec_dr = roster_df.groupby(['Secondary_employeeID'])['Date'].count().rename("sec_dr")
count_dr_shifts = pd.concat([count_pri_dr, count_sec_dr], axis=1).fillna(0)
count_dr_shifts['Total'] = count_dr_shifts.pri_dr + count_dr_shifts.sec_dr

count_dr_shifts = count_dr_shifts['Total'].reset_index().rename(columns={"index" : "Employee_ID", "Total" : "total_shifts"})   
sal_shifts_df = sal_df.merge(count_dr_shifts, how="left", on="Employee_ID")

sal_shifts_df['per_shift_pay'] = sal_shifts_df.total / sal_shifts_df.total_shifts
sal_shifts_df['per_shift_pay'] = sal_shifts_df.total / sal_shifts_df.total_shifts

sal_per_shift = sal_shifts_df[["Employee_ID", "per_shift_pay"]]

def pri_sal_cost(df):
    emp_id = df.Primary_employeeID
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

def sec_sal_cost(df):
    emp_id = df.Secondary_employeeID
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

roster_df['pri_sal'] = roster_df.apply(pri_sal_cost, axis=1)
roster_df['sec_sal'] = roster_df.apply(sec_sal_cost, axis=1)

roster_df['sal_cost'] = roster_df.pri_sal + roster_df.sec_sal

sal_df = roster_df[['Date','sal_cost','Primary_route', 'Satellite_Route_1','Satellite_Route_2']].fillna(0)    

def per_route_cost(df):
    count = 0 
    p_route = df.Primary_route
    route_1 = df.Satellite_Route_1
    route_2 = df.Satellite_Route_2
    if p_route != 0:
        count = count + 1
    if route_1 != 0:
        count = count + 1
    if route_2 != 0:
        routes = route_2.split("/")
        count = count + len(routes)
    cost = df.sal_cost / count
    return cost, count

sal_df[['per_run_sal', "count_of"]] = sal_df.apply(per_route_cost, axis=1, result_type='expand')
sal_df[sal_df.count_of > 1 ]


df_2 = sal_df[(sal_df.count_of.gt(1)) & (sal_df.count_of.lt(4))]

sal_df_1 = sal_df[sal_df.count_of.eq(1)][["Date", "Primary_route","per_run_sal","count_of"]].rename(columns={"Primary_route" : "route"})
sal_df_2 = df_2[["Date", "Primary_route","per_run_sal","count_of"]].rename(columns={"Primary_route" : "route"})
sal_df_3 = df_2[["Date", "Satellite_Route_1","per_run_sal","count_of"]].rename(columns={"Satellite_Route_1" : "route"})
sal_df_4 = df_2[["Date", "Satellite_Route_2","per_run_sal","count_of"]].rename(columns={"Satellite_Route_2" : "route"})

df_3 = sal_df[sal_df.count_of.gt(3)]
# df_3['per_run_sal'] =df_3.per_run_sal.astype(float)
df_3

sal_df

,Date,sal_cost,Primary_route,Satellite_Route_1,Satellite_Route_2,per_run_sal,count_of
0,2021-09-01,0.000000,RLC,0,0,0.000000,1.0
1,2021-09-01,619.383214,RLK,0,0,619.383214,1.0
2,2021-09-01,220.935714,BR1,0,0,220.935714,1.0
3,2021-09-01,328.550000,SWG,0,0,328.550000,1.0
4,2021-09-01,613.355000,HYG,RL5,RL6/RLP,153.338750,4.0
...,...,...,...,...,...,...,...
107,2021-09-07,288.940000,NIWPR1,0,0,288.940000,1.0
108,2021-09-07,431.345000,BR3,0,0,431.345000,1.0
109,2021-09-07,518.490952,RLD,0,0,518.490952,1.0
110,2021-09-07,518.490952,RLJ,0,0,518.490952,1.0


In [134]:
def isStr(df):
    if type(df['Satellite_Route_2']) == "str":
        x['Satellite_Route_2'].split('/')
        
(sal_df.set_index(['Date', 'per_run_sal', 'count_of'])
 .apply(isStr, axis=1).explode()
).reset_index().sort_values(0, ascending=True)


,Date,per_run_sal,count_of,0
0,2021-09-01,0.000000,1.0,None
1,2021-09-01,619.383214,1.0,None
2,2021-09-01,220.935714,1.0,None
3,2021-09-01,328.550000,1.0,None
4,2021-09-01,153.338750,4.0,None
...,...,...,...,...
107,2021-09-07,288.940000,1.0,None
108,2021-09-07,431.345000,1.0,None
109,2021-09-07,518.490952,1.0,None
110,2021-09-07,518.490952,1.0,None


In [116]:

# lambda x : x['Satellite_Route_2'].split('/') if x['Satellite_Route_2']  

,Date,sal_cost,Primary_route,Satellite_Route_1,Satellite_Route_2,per_run_sal,count_of
0,2021-09-01,0.000000,RLC,0,0,0.000000,1.0
1,2021-09-01,619.383214,RLK,0,0,619.383214,1.0
2,2021-09-01,220.935714,BR1,0,0,220.935714,1.0
3,2021-09-01,328.550000,SWG,0,0,328.550000,1.0
4,2021-09-01,613.355000,HYG,RL5,RL6/RLP,153.338750,4.0
...,...,...,...,...,...,...,...
107,2021-09-07,288.940000,NIWPR1,0,0,288.940000,1.0
108,2021-09-07,431.345000,BR3,0,0,431.345000,1.0
109,2021-09-07,518.490952,RLD,0,0,518.490952,1.0
110,2021-09-07,518.490952,RLJ,0,0,518.490952,1.0


In [105]:
# df_3[['Satellite_Route_2_a','Satellite_Route_2_b']]

# def 
.Satellite_Route_2.str.split('/')
#  .apply(lambda x: x[''].str.split('/'),axis=1)
 .explode('Satellite_Route_2')

<ipython-input-105-5cbdf61a23e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['Satellite_Route_2'] = (df_3


In [106]:
df_3

,Date,sal_cost,Primary_route,Satellite_Route_1,Satellite_Route_2,per_run_sal,count_of
4,2021-09-01,613.355,HYG,RL5,NaN,153.33875,4.0
92,2021-09-07,613.355,HYG,RL5,NaN,153.33875,4.0


In [114]:
df_2

,Date,sal_cost,Primary_route,Satellite_Route_1,Satellite_Route_2,per_run_sal,count_of
14,2021-09-01,369.597500,NIWPR2,CBK,0,184.798750,2.0
16,2021-09-01,305.250000,RED,RLR,0,152.625000,2.0
19,2021-09-01,1023.747500,UOSCB,CMDCB,0,511.873750,2.0
35,2021-09-02,602.442500,NIWPR2,CBK,0,301.221250,2.0
41,2021-09-02,305.250000,RED,RLR,0,152.625000,2.0
45,2021-09-02,421.305000,UOSCO,CMDCO,RLG,140.435000,3.0
47,2021-09-02,518.490952,RLD,RLJ,0,259.245476,2.0
65,2021-09-03,369.597500,NIWPR2,CBK,0,184.798750,2.0
66,2021-09-03,592.180714,RL9,BR2,0,296.090357,2.0
67,2021-09-03,305.250000,RLR,RED,0,152.625000,2.0


In [ ]:
# pd.concat(
#     [df_3(row['per_run_sal'], row['Satellite_Route_2'].split('/'))              
# for _, row in df_3.iterrows()
#     ]

# ).reset_index()
pd.concat([pd.DataFrame(
   data=[row.Date, row.per_run_sal, row.count_of,row.Satellite_Route_2.split('/')]
)for _, row in df_3.iterrows()])
# pd.concat(
#    [pd.DataFrame(
#        data=[row.Date, row.per_run_sal, row.count_of], 
#        index=row.Satellite_Route_2.split('/')
#    )for _, row in df_3.iterrows()] 
# )
# rint(row.per_run_sal, row.Satellite_Route_2.split('/'), row.count_of, idx)for idx, row in df_3.iterrows()]

In [31]:
sal_df_4

,Date,route,per_run_sal,count_of
14,2021-09-01,0,184.798750,2.0
16,2021-09-01,0,152.625000,2.0
19,2021-09-01,0,511.873750,2.0
35,2021-09-02,0,301.221250,2.0
41,2021-09-02,0,152.625000,2.0
45,2021-09-02,RLG,140.435000,3.0
47,2021-09-02,0,259.245476,2.0
65,2021-09-03,0,184.798750,2.0
66,2021-09-03,0,296.090357,2.0
67,2021-09-03,0,152.625000,2.0


In [11]:
pd.concat([sal_df_1,sal_df_2,sal_df_3,sal_df_4])

88400

In [ ]:
sal_df_2 = df_3[["Date", "Primary_route","per_run_sal","count_of"]].rename(columns={"Primary_route" : "route"})
sal_df_3 = df_3[["Date", "Satellite_Route_1","per_run_sal","count_of"]].rename(columns={"Satellite_Route_1" : "route"})
sal_df_4 = df_3[["Date", "Satellite_Route_2","per_run_sal","count_of"]].rename(columns={"Satellite_Route_2" : "route"})

In [138]:
# sal_df[sal_df.count_of.ne(1)]
df_2 = sal_df[(sal_df.count_of.ne(1)) & (sal_df.count_of.lt(4))]
sal_df_1 = sal_df[sal_df.count_of.eq(1)][["Date", "Primary_route","per_run_sal","count_of"]].rename(columns={"Primary_route" : "route"})    
sal_df_2 = df_2[["Date", "Primary_route","per_run_sal","count_of"]].rename(columns={"Primary_route" : "route"})
sal_df_3 = df_2[["Date", "Satellite_Route_1","per_run_sal","count_of"]].rename(columns={"Satellite_Route_1" : "route"})
sal_df_4 = df_2[["Date", "Satellite_Route_2","per_run_sal","count_of"]].rename(columns={"Satellite_Route_2" : "route"})


In [153]:
# sal_df_1

df_3 = sal_df[sal_df.count_of.gt(3)]
df_3.Satellite_Route_2.str.split("/").explode()

4     RL6
4     RLP
92    RL6
92    RLP
Name: Satellite_Route_2, dtype: object

In [143]:
sal_df_2[sal_df_2.route.ne(0)]

,Date,route,per_run_sal,count_of
14,2021-09-01,NIWPR2,184.798750,2.0
16,2021-09-01,RED,152.625000,2.0
19,2021-09-01,UOSCB,511.873750,2.0
35,2021-09-02,NIWPR2,301.221250,2.0
41,2021-09-02,RED,152.625000,2.0
45,2021-09-02,UOSCO,140.435000,3.0
47,2021-09-02,RLD,259.245476,2.0
65,2021-09-03,NIWPR2,184.798750,2.0
66,2021-09-03,RL9,296.090357,2.0
67,2021-09-03,RLR,152.625000,2.0


In [141]:
sal_df_3

,Date,route,per_run_sal,count_of
14,2021-09-01,CBK,184.798750,2.0
16,2021-09-01,RLR,152.625000,2.0
19,2021-09-01,CMDCB,511.873750,2.0
35,2021-09-02,CBK,301.221250,2.0
41,2021-09-02,RLR,152.625000,2.0
45,2021-09-02,CMDCO,140.435000,3.0
47,2021-09-02,RLJ,259.245476,2.0
65,2021-09-03,CBK,184.798750,2.0
66,2021-09-03,BR2,296.090357,2.0
67,2021-09-03,RED,152.625000,2.0


In [144]:
sal_df_4[sal_df_4.route.ne(0)]

,Date,route,per_run_sal,count_of
45,2021-09-02,RLG,140.435,3.0


In [67]:
roster_df[~roster_df.Satellite_Route_2.isnull()]


,Date,Run_type,Primary_employeeID,Secondary_employeeID,Primary_driver_Name,Secondary_Driver_Name,Primary_route,Satellite_Route_1,Satellite_Route_2,Primary_truck,Alternative_Truck,Subcontracted_From/Special_Client,Note,pri_sal,sec_sal,sal_cost
4,2021-09-01,DAY RUNS,swlocbra,NaN,BRAD LOCKE,NaN,HYG,RL5,RL6/RLP,CF 71 JM,NaN,NaN,NaN,613.355,0.0,613.355
45,2021-09-02,DAY RUNS,swwarrob,NaN,ROB WARD,NaN,UOSCO,CMDCO,RLG,CQ 64 LO,NaN,UNIVERSITY,NaN,421.305,0.0,421.305
92,2021-09-07,DAY RUNS,swlocbra,NaN,BRAD LOCKE,NaN,HYG,RL5,RL6/RLP,CF 71 JM,NaN,NaN,NaN,613.355,0.0,613.355


['e']

In [ ]:

# emp_id = df.Secondary_employeeID
# sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values


# .T.fillna(0)

# roster_df.Satellite_Route_2


In [52]:
roster_df

,Date,Run_type,Primary_employeeID,Secondary_employeeID,Primary_driver_Name,Secondary_Driver_Name,Primary_route,Satellite_Route_1,Satellite_Route_2,Primary_truck,Alternative_Truck,Subcontracted_From/Special_Client,Note,pri_sal
0,2021-09-01,NIGHT RUNS,subcon1,NaN,ALAN WALDEN,NaN,RLC,NaN,NaN,CQ 64 LO,NaN,NaN,NaN,0.000000
1,2021-09-01,DAY RUNS,swpetand,swhearay,ANDY,RAY,RLK,NaN,NaN,XO62AB,XN71FD,NaN,NaN,220.935714
2,2021-09-01,DAY RUNS,swpetand,NaN,ANDY PETERSON,NaN,BR1,NaN,NaN,XN 75 OS,NaN,NaN,NaN,220.935714
3,2021-09-01,DAY RUNS,swedwben,NaN,BEN EDWARDS,NaN,SWG,NaN,NaN,XN 72 MK,NaN,NaN,NaN,328.550000
4,2021-09-01,DAY RUNS,swlocbra,NaN,BRAD LOCKE,NaN,HYG,RL5,RL6/RLP,CF 71 JM,NaN,NaN,NaN,613.355000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2021-09-07,DAY RUNS,swliqi,NaN,RICKY LI,NaN,NIWPR1,NaN,NaN,XPL 658,NaN,VISY,NaN,288.940000
108,2021-09-07,DAY RUNS,swmittai,NaN,TAIT MITCHELL,NaN,BR3,NaN,NaN,XN 06 OJ,NaN,NaN,NaN,431.345000
109,2021-09-07,DAY RUNS,swsinvic,ir12,VICTOR,LUKE,RLD,NaN,NaN,CL 02 TB,NaN,NaN,NaN,283.484286
110,2021-09-07,DAY RUNS,swsinvic,ir12,VICTOR,LUKE,RLJ,NaN,NaN,CL O2 TB,NaN,NaN,NaN,283.484286


In [48]:
count_dr_shifts

,pri_dr,sec_dr,Total
ir02,3.0,1.0,4.0
ir12,2.0,4.0,6.0
irdeaz,5.0,0.0,5.0
subcon1,3.0,0.0,3.0
swansgia,5.0,0.0,5.0
swbranat,5.0,1.0,6.0
swbraric,4.0,0.0,4.0
swburlei,4.0,0.0,4.0
swdinvie,5.0,0.0,5.0
swedwben,5.0,0.0,5.0


In [54]:
count_dr_shifts = count_dr_shifts['Total']

KeyError: 'Total'

In [45]:
# count_dr_shifts = count_dr_shifts.rename('total_shifts').reset_index().rename(columns={"index" : "Employee_ID"})

# .reset_index().rename(columns={"index" : "Employee_ID"})